# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [ ]:
import pandas as pd
import numpy as np

recipes = pd.read_csv("recipes.tsv", sep="\t")
recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")

favor_keywords = {"eggs", "cheese", "healthy", "fruit", "sushi", "ramen", "hearty", "avocado", "vegetarian", "italian", "salsa", "burger", "tomatoes", "spicy", "garlic", "beef", "chicken"}
disfavor_keywords = {"spam", "bacon", "dessert", "fudgy", "chocolate", "frosting", "sugar", "pie"}

tag_counts = recipe_tags.groupby("recipe_slug")["recipe_tag"].apply(list)

def assign_raw_rating(tags):
    tags = set(tags)
    score = 3  
    if tags & favor_keywords:
        score += 1
    if tags & disfavor_keywords:
        score -= 1
    return min(max(score, 1), 5)

recipes["rating_raw"] = recipes["recipe_slug"].map(lambda slug: assign_raw_rating(tag_counts.get(slug, [])))

recipes.loc[:9, "rating_raw"] = 1   
recipes.loc[10:19, "rating_raw"] = 5  

recipes["rating"] = (recipes["rating_raw"] - 1) * 0.25

ratings = recipes[["recipe_slug", "rating"]]
ratings.to_csv("ratings.tsv", sep="\t", index=False)


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [29]:
# YOUR CHANGES HERE

recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")

ratings = pd.read_csv("ratings.tsv", sep="\t")

tag_matrix = (
    recipe_tags
    .assign(value=1)
    .pivot_table(index="recipe_slug", columns="recipe_tag", values="value", fill_value=0)
    .reset_index()
)

tag_matrix["bias"] = 1

cols = ["recipe_slug", "bias"] + sorted([col for col in tag_matrix.columns if col not in ["recipe_slug", "bias"]])
features = tag_matrix[cols]

features = ratings[["recipe_slug"]].merge(features, on="recipe_slug", how="left")

assert (features["recipe_slug"] == ratings["recipe_slug"]).all()

features.to_csv("features.tsv", sep="\t", index=False)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [46]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

features = pd.read_csv("features.tsv", sep="\t").sort_values("recipe_slug").reset_index(drop=True)
ratings = pd.read_csv("ratings.tsv", sep="\t").sort_values("recipe_slug").reset_index(drop=True)

features = features.sort_values("recipe_slug").reset_index(drop=True)
ratings = ratings.sort_values("recipe_slug").reset_index(drop=True)

X = features.drop(columns=["recipe_slug"])

y = ratings["rating"]

model = Ridge(alpha=1.0, fit_intercept=False)

model.fit(X, y)

predictions = model.predict(X)

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

print(f"Model trained. MSE: {mse:.4f}, R²: {r2:.4f}")


Model trained. MSE: 0.0045, R²: 0.9448


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE


model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)

feature_names = X.columns

coefficients = model.coef_

model_df = pd.DataFrame({
    "recipe_tag": feature_names,
    "coefficient": coefficients
})

model_df.to_csv("model.tsv", sep="\t", index=False)


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [48]:
# YOUR CHANGES HERE

features = pd.read_csv("features.tsv", sep="\t").sort_values("recipe_slug").reset_index(drop=True)
ratings = pd.read_csv("ratings.tsv", sep="\t").sort_values("recipe_slug").reset_index(drop=True)

X = features.drop(columns=["recipe_slug"])
recipe_slugs = features["recipe_slug"]

y = ratings["rating"]
model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)

score_estimates = model.predict(X)

estimates_df = pd.DataFrame({
    "recipe_slug": recipe_slugs,
    "score_estimate": score_estimates
})

estimates_df.to_csv("estimates.tsv", sep="\t", index=False)


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [34]:
# YOUR CHANGES HERE
import numpy as np


features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df = pd.read_csv("ratings.tsv", sep="\t")

X = features_df.drop(columns=["recipe_slug"]).values 
y = ratings_df["rating"].values                       


lambda_ = 1
alpha = 2


A = X.T @ X + lambda_ * np.identity(X.shape[1])  
b = X.T @ y                                     

A_inv = np.linalg.inv(A)


theta_hat = A_inv @ b  


score_bounds = []
for x in X:
    x = x.reshape(-1, 1)  
    mean_estimate = float((theta_hat @ x)[0])
    uncertainty = float(np.sqrt((x.T @ A_inv @ x)[0,0]))
    bound = mean_estimate + alpha * uncertainty
    score_bounds.append(bound)


bounds_df = pd.DataFrame({
    "recipe_slug": features_df["recipe_slug"],
    "score_bound": score_bounds
})
bounds_df.to_csv("bounds.tsv", sep="\t", index=False)


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [35]:
# YOUR CHANGES HERE

features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df = pd.read_csv("ratings.tsv", sep="\t")

X_all = features_df.drop(columns=["recipe_slug"]).values  
recipe_slugs = features_df["recipe_slug"].tolist()
ratings = ratings_df.set_index("recipe_slug")["rating"].to_dict()

lambda_ = 1
alpha = 2
n_features = X_all.shape[1]

A = lambda_ * np.identity(n_features)
b = np.zeros((n_features,))

recommended = set()

recommendation_log = []

for t in range(100):
    best_bound = -np.inf
    best_index = None
    best_x = None

    A_inv = np.linalg.inv(A)
    theta_hat = A_inv @ b

    for i, (slug, x) in enumerate(zip(recipe_slugs, X_all)):
        if slug in recommended:
            continue

        x_vec = x.reshape(-1, 1)
        mean = float(theta_hat @ x)
        uncertainty = float(np.sqrt((x_vec.T @ A_inv @ x_vec) [0,0]))
        bound = mean + alpha * uncertainty

        if bound > best_bound:
            best_bound = bound
            best_index = i
            best_x = x

    chosen_slug = recipe_slugs[best_index]
    reward = ratings[chosen_slug]

    x = best_x.reshape(-1, 1)
    A += x @ x.T
    b += reward * best_x

    recommendation_log.append({
        "recipe_slug": chosen_slug,
        "score_bound": best_bound,
        "reward": reward
    })

    recommended.add(chosen_slug)

recommendations_df = pd.DataFrame(recommendation_log)
recommendations_df.to_csv("recommendations.tsv", sep="\t", index=False)


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [39]:
acknowledgements = [
['1. https://github.com/BartyzalRadek/contextual-bandits-recommender/blob/master/linUCB.py', 
 'To learn more about linUCB code especially the vectors and alorgithm behind it.'],
['2. https://github.com/umeshksingla/news-recommend-ire/blob/master/src/imbalanced_classification.py',
 'To look at repos that implements LinUCB'],
 ['3. https://github.com/niffler92/Bandit/blob/master/bandit/bandit.py', 'To learn more about contextual bandits'],
 ['4. https://wensun.github.io/CS6789_data/linUCB.pdf', 'To learn more about the formulas discuss in class about the linUCB Linear Upper Confidence Bound']
]

columns = ['Source', 'Used For/Reason']

ack_df = pd.DataFrame(acknowledgements, columns=columns)

ack_df.to_csv("acknowledgements.txt", sep="\t", index=False)

Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.